In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import creep_event_picker as cep
import obspy
from matplotlib.dates import num2date
import matplotlib.dates as mdates
%matplotlib qt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def check_dir(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path, exist_ok=True)

Import creepmeter database

In [3]:
Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
Creepmeter_dataframe

,Network,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude,Length,Depth,Obliquity,Creepmeter_full_name
0,EAF,BAL1,BAL1_1T,2023-03-12T10:47:55.000000Z,2023-09-05T13:14:55.000000Z,0.016667,1T,37.9903,38.1990,5 m,NaN,30,Balikburnu
1,PARK,C461,C461_10T,2004-09-29T02:20:00.000000Z,2022-06-20T12:40:00.000000Z,0.001667,10T,35.7240,-120.2820,NaN,NaN,30,Highway 46
2,PARK,C462,C462_1T,2021-08-19T19:10:11.000000Z,2024-03-22T01:05:11.000000Z,0.016667,1T,35.7240,-120.2820,NaN,NaN,30,Highway 46
3,UTA,CAN1,CAN1_30T,2012-06-12T15:33:00.000000Z,2017-06-15T08:33:00.000000Z,0.000556,30T,38.0198,-110.0374,7.5 m,NaN,45,Canyonlands
4,NAF,CER1,CER1_30S,2019-10-09T06:46:00.000000Z,2024-05-23T10:25:30.000000Z,0.033333,30S,40.8947,32.7773,20 m,NaN,29,Cerkes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,HOL,XSJ3,XSJ3_10T,2007-02-15T20:19:59.000000Z,2016-12-08T18:09:59.000000Z,0.001667,10T,36.8360,-121.5210,20 m,NaN,30,San Juan Bautista #3
121,PARK,XTA1,XTA1_10T,1985-12-18T00:00:00.000000Z,2024-10-09T18:50:00.000000Z,0.001667,10T,35.8900,-120.4270,30 m,1 m,30,Taylor Ranch
122,PARK,XVA1,XVA1_10T,1987-05-29T00:00:00.000000Z,2017-10-23T22:10:00.000000Z,0.001667,10T,35.9220,-120.4620,30 m,1 m,30,Varian
123,EAF,YZE1,YZE1_1T,2023-09-07T10:36:07.000000Z,2024-05-26T12:59:07.000000Z,0.016667,1T,38.1791,38.7526,20 m,NaN,30,Yazica East


In [8]:
plt.close('all')
for i in range(2,3):#len(Creepmeter_dataframe)):
    df_starts = pd.DataFrame(columns=['ST'])
    path = "../../Data/DATA_tidied/SAC/{q}.SAC".format(q =Creepmeter_dataframe.File_code.iloc[i])
    path_save = "../../Data/DATA_tidied/Picks/"
    check_dir(path_save)
    print(path)
    st = obspy.read(path)
    tr = st[0]
    tr.interpolate(sampling_rate=tr.stats.sampling_rate)
    tm = tr.times("utcdatetime")
    creep = tr.data
    sample_rate =  round(tr.stats.delta)/60
    tm_dt = []
    for w in range(len(tm)):
        tm_new = tm[w].datetime
        tm_dt.append(tm_new)
    print('now interpolate')
    tm_int, creep_int, upsampled = cep.interpolate(tm_dt,creep,sample_rate)
    
    creep_DT = scipy.signal.detrend(creep_int)
    upper_freq=7200#input("longer duration for bandpass filtering in mins, e.g., 5 days = 7200")
    lower_freq=120#input("shorter duration for bandpass filtering in mins, e.g., 2 hrs =  120")
    sos = scipy.signal.butter(4,[1/int(upper_freq),1/int(lower_freq)], 'band',output = 'sos',fs=0.10) #bandpass filter for 2hrs and 5days
    creep_data  = scipy.signal.sosfiltfilt(sos,creep_DT) # filter the data


    fig = plt.figure()
    ax = plt.subplot(1,1,1)
    plt.plot(tm_int,creep_int,marker='o', color='b', linestyle='-', linewidth=2, markersize=6)
    df_picks = pd.read_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =Creepmeter_dataframe.File_code.iloc[i]),index_col=0)
    for j in range(len(df_picks)):
        boolarr = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None),\
                                            np.array(tm_int)<=pd.to_datetime(df_picks.ET.iloc[j]).replace(tzinfo=None))
        plt.plot(tm_int[boolarr],creep_int[boolarr])
    #df_picks = pd.DataFrame()
    
    scale = 1.1
    zp = cep.ZoomPan()
    figZoom = zp.zoom_factory(ax, base_scale = scale)
    figPan = zp.pan_factory(ax)
    s = 1
    fig.set_size_inches(11,11)
    fig.tight_layout()
    plt.show()
    pts = plt.ginput(200,timeout = -1)
    plt.close('all')
    for sublist in pts:
        df = pd.DataFrame({'ST':[sublist[0]],'SD':[sublist[1]]})
        df_starts = pd.concat([df_starts,df])
    

    plt.show()
    if len(df_starts) >0:
        #df_picks.reset_index(inplace=True)     
        for r in range(len(df_starts)):
            time_convert = mdates.num2date(df_starts.ST.iloc[r])
            s3 = pd.Series(time_convert).dt.round("1min")
            df_starts.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)   
        
        df_starts['ST'] = df_starts['ST'].astype(str).str[0:19]
        df_starts.ST = pd.to_datetime(df_starts.ST)
        df_starts.sort_values(by='ST',inplace=True)
        
        plt.close('all')
        print(df_starts)
        ##### define end ####
        
        df_end=pd.DataFrame(columns=['ET'])
        fig = plt.figure()
        ax = plt.subplot(1,1,1)
        plt.plot(tm_int,creep_int)
        for j in range(len(df_picks)):
            boolarr = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_picks.ST_new.iloc[j]).replace(tzinfo=None),\
                                            np.array(tm_int)<=pd.to_datetime(df_picks.ET_new.iloc[j]).replace(tzinfo=None))
            event_creep = creep_int[boolarr]
            event_time = tm_int[boolarr]
            ax.plot(event_time,event_creep,color='orange')
        plt.scatter(pd.to_datetime(df_starts.ST),df_starts.SD,c='red')
        for j in range(len(df_starts)):
            ax.text(pd.to_datetime(df_starts.ST.iloc[j]),df_starts.SD.iloc[j],'{k}/{n}'.format(k=df_starts.index[j]+1,n=len(df_starts)))
        scale = 1.1
        zp = cep.ZoomPan()
        figZoom = zp.zoom_factory(ax, base_scale = scale)
        figPan = zp.pan_factory(ax)
        s = 1
        fig.set_size_inches(11,11)
        #fig.tight_layout()
        plt.show()
        for j in range(len(df_starts)):
            print('{m}/{n}'.format(m=df_starts.index[j]+1,n=len(df_starts)))
            pts = plt.ginput(1,timeout = -1)
            
            for sublist in pts:
                df = pd.DataFrame({'ET':[sublist[0]]})
                df_end = pd.concat([df_end,df]) 
        plt.close()

        for r in range(len(df_end)):
            time_convert = mdates.num2date(float(df_end.ET.iloc[r]))
            s3 = pd.Series(time_convert).dt.round("1min")
            df_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)  
        
        #df_picks_end.ET= mdates.num2date(df_picks_end.ET)
        df_end.reset_index(inplace=True)
        df_end.drop(columns=['index'],inplace=True)
        
        df_starts.reset_index(inplace=True)
        df_starts['ET_new'] = df_end['ET']
        df_starts['Quality_check'] = ['CE']*len(df_starts)
        df_starts.drop(['index','SD'],axis=1,inplace=True)
        df_picks = pd.concat([df_picks,df_starts])
        df_picks['ST_new'] = df_picks['ST_new'].astype(str).str[0:19]
        df_picks['ET_new'] = df_picks['ET_new'].astype(str).str[0:19]
        print(df_picks)
        df_picks.to_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =Creepmeter_dataframe.File_code.iloc[i]))

../../Data/DATA_tidied/SAC/C462_1T.SAC
now interpolate


KeyboardInterrupt: 

In [6]:
df_picks

,ST,ET
0,2021-08-19 20:18:00,2021-08-20 22:45:00
0,2021-09-29 08:58:00,2021-09-29 23:08:00
0,2021-10-13 12:24:00,2021-10-14 00:14:00
0,2021-10-20 05:12:00,2021-10-22 09:38:00
0,2021-11-22 14:21:00,2021-11-24 21:23:00
0,2021-11-30 11:50:00,2021-12-03 08:15:00
0,2021-12-06 13:09:00,2021-12-07 11:32:00
0,2021-12-11 16:30:00,2021-12-12 08:00:00
0,2021-12-12 11:50:00,2021-12-13 06:14:00
0,2022-01-28 11:22:00,2022-01-30 17:08:00


In [ ]:
df_picks.to_csv("../../../Data/{p}/Picks/{q}_picks.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
                                                                    q =Creepmeter_dataframe.File_code.iloc[i]))

In [ ]:
df_starts.reset_index(inplace=True)

In [ ]:
df_starts.to_csv("../../../Data/{p}/Picks/{q}_picks_starts.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
                                                                    q =Creepmeter_dataframe.File_code.iloc[i]))

In [6]:
df_picks

,ST,ET,Quality_check,ET_new,ST_new,Quality_check_new
0,2018-09-14 03:55:00,2018-09-14 06:21:00,NCE,2018-09-14 06:21:00,2018-09-14 03:55:00,NCE
1,2018-09-14 07:30:00,2018-09-14 15:00:00,CE,2018-09-15 11:14:00,2018-09-14 07:30:00,CE
2,2018-10-08 08:57:00,2018-10-08 09:51:00,NCE,2018-10-09 17:28:00,2018-10-08 08:57:00,CE
3,2018-10-08 10:19:00,2018-10-08 11:19:00,NCE,2018-10-08 11:19:00,2018-10-08 10:19:00,NCE
4,2018-10-08 13:39:00,2018-10-08 14:22:00,NCE,2018-10-08 14:22:00,2018-10-08 13:39:00,NCE
...,...,...,...,...,...,...
179,2022-12-02 14:18:00,2022-12-03 04:29:00,CE,2022-12-04 15:03:00,2022-12-02 14:18:00,CE
180,2022-12-22 03:29:00,2022-12-22 14:33:00,CE,2022-12-27 02:49:00,2022-12-22 03:29:00,CE
181,2023-02-08 23:58:00,2023-02-09 06:41:00,NCE,2023-02-14 15:44:00,2023-02-08 23:58:00,CE
182,2023-02-24 08:01:00,2023-02-24 10:45:00,NCE,2023-02-24 10:45:00,2023-02-24 08:01:00,CE


In [ ]:
i=0

In [ ]:
df_fixer = pd.read_csv("../../../Data/{p}/Picks/{q}_picks.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
                                                                    q =Creepmeter_dataframe.File_code.iloc[i]),index_col=0)

In [ ]:
df_fixer

In [ ]:
st_date = df_fixer.ST.astype(str).str[0:10]
st_time  = df_fixer.ST.astype(str).str[11:]
df_fixer.ST = st_date+ ' ' +st_time

In [ ]:
st_date = df_fixer.ET.astype(str).str[0:10]
st_time  = df_fixer.ET.astype(str).str[11:]
df_fixer.ET = st_date+ ' ' +st_time

In [ ]:
df_fixer

In [ ]:
df_fixer.to_csv("../../../Data/{p}/Picks/{q}_picks.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
                                                                    q =Creepmeter_dataframe.File_code.iloc[i]))

In [ ]:
for i in range(4,)